In [1]:
from operator import *
import itertools as it
import numpy as np
import time
import copy
import os

In [2]:
# Testing how to do Cartesian products of lists
# This will be needed later to stitch together lists of sides to get a full corona
A=[[1]]
B=[[2]]
C=[[3]]
D=[[4],[5]]
X = it.product(A,B,C,D)
for i in X:
    print(i)

([1], [2], [3], [4])
([1], [2], [3], [5])


# Side Type Generators

In [3]:
# C = size of central square
# m = maximum size square (square sizes are 1, 2, ..., m)
# A side contains squares of sizes a0, a1, ..., ak
# A side is denoted by a0^e.a1.a2. ... . ak where the exponent e on a0 is the amount of "prehang" 
# that the first square a0 has with the central square.
# In general, e satisfies 0 <= e <= a0.
# The data type for a side is [e,[a0,a1,...,ak]]

## Side Type i

In [4]:
# Side type i (afterhang)
# Type i has the following constraints:

# exponent constraint: 
# e = 0
 
# Partition Constraints:
# Because the last square ak must overhang, we must have
# C + 1 <= sum(ai,(i,0,k)) <= (C + m) - 1
# This tells us that we want to find integer partitions of 
# C+1, C+2, ..., and (C + m) - 1 to get the side sequences a0,a1,...,ak

# Overlap Constraints:
# The last square ak must share a nonzero edge length with the central square C,
# but ak should not perfectly align with the corner of the central square C
# This gives rise to the numerical constraint on the sides:
# sum(ai,(i,0,k-1)) < C < sum(ai,(i,0,k))
# This will serve as a filter to weed out partitions a0,a1,...,ak that don't overhang properly

# Other filters:
# (1) Remove instances where a0 = C (violates unilaterality since e = 0 in this case)
# (2) Remove instances where ai = a(i+1)
# (3) Remove instances where the partition contains "a,1,b" or "a,2,b" (violates vortex condition for 1- and 2-squares)
# (4) Apply overlap conditions as a filter (these require summing so should come last)

# Outline of algorithm:
# inputs: C = size of center square, m = max size square
# For each value of n = C+1, C+2, ... (C + m) - 1:
#     Let P(n) = List of all integer partitions of n (and their distinct permutations)
#     where the partitions use only integers 1, 2, ..., m
#     Let FilteredP(n) = P(n)
#     For each L in P(n):
#         Apply various filters to detect if L should be removed from P(n).
#             If L fails a filter:
#                 FilteredP(n).remove(L)
#
#     return FilteredP(n)

def Side_Type_i_Generator(C,m):

    #Using integer partitions to create a list of all potential sides of the form [0,[a0,a1,...,ak]]
    AllPotentialSides = []
    for n in range(C+1,C+m):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            for x in MoreParts:
                AllPotentialSides.append([0,x]) #the exponent is 0 for all sides of type 
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where a0 = C
        if L[1][0] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the first square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions
        # We must have sum(ai,(i,0,k-1)) < C < sum(ai,(i,0,k))
        # Remove any L that does not satisfy that condition
        
        Lsum1 = sum(L[1][i] for i in range(len(L[1])-1))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)
  
        if Lsum1 >= C or C >= Lsum2:
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraints")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [5]:
Side_Type_i_Generator(1,4)

[[0, [2]], [0, [3]], [0, [4]]]

## Side Type ii

In [6]:
# Side type ii (prehang)
# Type ii has the following constraints:

# exponent constrait: 
# 0 < e <= a0 and e is an integer

# partition constraint:
# Because the first square a0 must overhang, we must have
# C + 1 <= sum(ai,(i,0,k)) <= (C + m)
# This tells us that we want to find integer partitions of 
# C+1, C+2, ..., and (C + m) to get the side sequences a0,a1,...,ak 

# overlap constraint 1:
# e = sum(ai,(i,0,k)) - C
# This determines the exponent e and we can check against
# the exponent constraint to filter out bad partitions a0,a1,...,ak

# overlap constraint 2:
# sum(ai,(i,1,k)) <= C < sum(ai,(i,0,k))
# This can be used to filter out partitions that cannot overlap properly
# at the a0 square.

# Other filters:
# (1) Remove instances where ak = C (violates unilaterality since e = 0 in this case)
# (2) Remove instances where ai = a(i+1)
# (3) Remove instances where the partition contains "a,1,b" or "a,2,b" (violates vortex condition for 1- and 2-squares)
# (4) Apply overlap conditions as a filter (these require summing so should come last)

def Side_Type_ii_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+1, C+2, ..., and (C + m)
    AllPotentialSides = []
    for n in range(C+1,C+m+1):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            # Adding exponents
            for x in MoreParts:
                # The exponent constraint is 0 < e <= a0 and e is an integer
                a0 = x[0]
                for e in range(1,a0+1):
                    AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where ak = C
        if L[1][len(L[1])-1] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the last square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions.
        # Constraint 1: e = sum(ai,(i,0,k)) - C
        # Constraint 2: sum(ai,(i,1,k)) <= C < sum(ai,(i,0,k))
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(1,len(L[1])))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint #1:
        if L[0] != (Lsum2 - C):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #1")
            continue

        # Checking overlap constraint #2
        if Lsum1 > C or C >= Lsum2:
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)

        # Removing the corner case: 
        # if the side is with e = a0, then remove it
        if (L[0] == L[1][0]):
            FilteredSides.remove(L)
            #print("This L got filtered since it is a corner case with e = a0")
            continue 
    
    
    #print(FilteredSides)
    
    return(FilteredSides)


In [7]:
Side_Type_ii_Generator(4,4)

[[1, [4, 1]], [1, [3, 2]], [1, [2, 3]], [2, [4, 2]], [3, [4, 3]]]

## Side Type iii

In [8]:
# Side type iii (nohang)
# Type iii has the following constraints:


# exponent constrait: e = 0.

# Partition constraint:
# sum(ai,(i,0,k)) = C
# We must simply find all partitions of C

def Side_Type_iii_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C.
    AllPotentialSides = []
           
    #Generate all partitions of C using integers 1, 2, ..., m
    P = Partitions(C,max_part = m).list()
    #print(P)
        
    #Creating all permutations of the integer partitions
    for BasePart in P:
        MoreParts = Permutations(BasePart) 
        # Adding exponents
        for x in MoreParts:
            # The exponent constraint is e = 0
            AllPotentialSides.append([0,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where a0 = C
        if L[1][0] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the first square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [9]:
Side_Type_iii_Generator(4,4)

[[0, [3, 1]], [0, [1, 3]]]

## Side Type iv

In [10]:
# Side type iv (doublehang)
# Type iv has the following constraints:

# exponent constrait: 0 < e <= a0. In certain special cases e may be a continuous parameter,
# but most times it must be an integer due to geometric constraints

# partition constraint:
# Both a0 and ak must overhang.
# The overhang of a0 can be as much as m.
# The overhang of ak can be as much as m-1.
# C + 2 <= sum(ai,(i,0,k)) <= (C + 2m - 1)
# This tells us that we want to find integer partitions of 
# C+2, C+3, ..., and (C + 2m - 1) to get the side sequences a0,a1,...,ak 

# Overlap constraints
# The last square ak must overhang but not align with the corner.
# This gives rise to the following:
# sum(ai,(i,0,k-1)) - e < C < sum(ai,(i,0,k)) - e
# This will serve as a filter on partitions together with e.

def Side_Type_iv_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+2, C+3, ..., and (C + 2m - 1)
    AllPotentialSides = []
    for n in range(C + 2 , C + 2*m):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            
            # Adding integer exponents
            # Note: Later need to analyze which partitions should have a continuous parameter for the exponent.
            for x in MoreParts:
                # The exponent constraint is 0 < e <= a0 and e is an integer
                a0 = x[0]
                for e in range(1,a0+1):
                    AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        # Using the overlap constraints to filter out partitions.
        # Constraint: sum(ai,(i,0,k-1)) - e < C < sum(ai,(i,0,k)) - e
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(0,len(L[1])-1))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint
        if (Lsum1 - L[0]) >= C or C >= (Lsum2 - L[0]):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue
        #else:
        #    print("This L survived")
        #    print("C = ", C)
        #    print("Lsum1 = ", Lsum1)
        #    print("Lsum2 = ", Lsum2)

        # Removing the corner case: 
        # if the side is with e = a0, then remove it
        if (L[0] == L[1][0]):
            FilteredSides.remove(L)
            #print("This L got filtered since it is a corner case with e = a0")
            continue 
    
    #print(FilteredSides)
    
    return(FilteredSides)

In [11]:
Side_Type_iv_Generator(4,4)

[[1, [4, 2]], [1, [2, 4]], [1, [4, 3]], [2, [4, 3]], [1, [3, 4]], [2, [3, 4]]]

## Side Type v

In [12]:
# Side Type v generator: (Corner case of prehang)
#Side type v will be just like side type ii with the restriction that the exponent e is equal to a0 (the first square size).
def Side_Type_v_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+1, C+2, ..., and (C + m)
    AllPotentialSides = []
    for n in range(C+1,C+m+1):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            # Adding exponents
            for x in MoreParts:
                # The exponent constraint is e = a0
                a0 = x[0]
                e = a0
                AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)

        #Remove any instances where ak = C
        if L[1][len(L[1])-1] == C:
            FilteredSides.remove(L)
            #print("This L got filtered since the last square equals the center (unilaterality check)")
            continue
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue

        #add a filter where the first square cannot be a size 1 or 2
        #for example, [2,[2,3,4]] is not allowed since the first square is a 2
        if (L[1][0] == 1) or (L[1][0] == 2):
            FilteredSides.remove(L)
            #print("This L got filtered since the first square is a 1 or 2")
            continue

        # Using the overlap constraints to filter out partitions.
        # Constraint 1: e = sum(ai,(i,0,k)) - C
        # Constraint 2: sum(ai,(i,1,k)) <= C < sum(ai,(i,0,k))
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(1,len(L[1])))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint #1:
        if L[0] != (Lsum2 - C):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #1")
            continue

        # Checking overlap constraint #2
        if Lsum1 > C or C >= Lsum2:
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue

    return(FilteredSides)

In [13]:
Side_Type_v_Generator(4,4)

[[4, [4, 3, 1]]]

## Side Type vi

In [14]:
#Side Type vi generator:
#Side type vi will be just like side type iv with the restriction that the exponent e is equal to a0 (the first square size).
def Side_Type_vi_Generator(C,m):

    # Using integer partitions to create a list of all potential sides of the form [e,[a0,a1,...,ak]]
    # The partition constraint tells us to find partitions of C+2, C+3, ..., and (C + 2m - 1)
    AllPotentialSides = []
    for n in range(C + 2 , C + 2*m):
        
        #Generate all partitions of n using integers 1, 2, ..., m
        P = Partitions(n,max_part = m).list()
        #print(P)
        
        #Creating all permutations of the integer partitions
        for BasePart in P:
            MoreParts = Permutations(BasePart) 
            
            # Adding integer exponents
            # Note: Later need to analyze which partitions should have a continuous parameter for the exponent.
            for x in MoreParts:
                # The exponent constraint is e = a0
                a0 = x[0]
                e = a0
                AllPotentialSides.append([e,x])
                
       
    # Filtered Sides is intialized as a copy of AllPotentialSides.
    # We will remove sides from FilteredSides when the are caught by a filter.
    FilteredSides = []
    for i in AllPotentialSides:
        FilteredSides.append(i)
    
    for L in AllPotentialSides:

        #print("\n L = ",L)
            
        #Remove any instances where L has ai = a(i+1) for some i = 0, ..., n-2
        flag = False
        for i in range(len(L[1])-1):
            if L[1][i] == L[1][i+1]:
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since the consecutive sides are equal")
            continue

        #Remove any instances where L contains "a,1,b" or "a,2,b" since any such instance
        #violates the vortex condition for size-1 and size-2 squares
        flag = False
        for i in range(1,len(L[1])-1):
            if (L[1][i] == 1) or (L[1][i] == 2):
                flag = True
                break
        if flag == True:
            FilteredSides.remove(L)
            #print("This L got filtered since there was a 1 or 2 in the middle")
            continue
        
        #Remove any sides where the first square is a size 1 or 2
        if (L[1][0] == 1) or (L[1][0] == 2):
            FilteredSides.remove(L)
            #print("This L got filtered since the first square is a 1 or 2")
            continue
        

        # Using the overlap constraints to filter out partitions.
        # Constraint: sum(ai,(i,0,k-1)) - e < C < sum(ai,(i,0,k)) - e
        # Remove any L that does not satisfy these conditions
        
        Lsum1 = sum(L[1][i] for i in range(0,len(L[1])-1))
        Lsum2 = sum(L[1][i] for i in range(len(L[1])))
        #print("Lsum1 = ", Lsum1)
        #print("Lsum2 = ", Lsum2)

        # Checking overlap constraint
        if (Lsum1 - L[0]) >= C or C >= (Lsum2 - L[0]):
            FilteredSides.remove(L)
            #print("This L got filtered by overlap constraint #2")
            continue
        
    return(FilteredSides)


In [15]:
Side_Type_vi_Generator(4,4)

[[4, [4, 3, 2]], [4, [4, 3, 4]]]

# Corona Filters

## Filters on the Corona Itself

In [16]:
#Corona Filters:
# These are filters that will be applied to a full corona after stitching together four sides.

#1. Unilaterality check: When the sides meet they must not create a situation where two squares of the same size align perfectly at a corner.
#This can be checked by looking at the last square of one side and the first square of the next side. Note that the next side is either type v or vi.
#If they are the same size, then this violates unilaterality.
#Make sure that it also checks the last side connecting to the first side.

def Corona_Unilaterality_Check(Corona):
    #Corona is of the form [Side1, Side2, Side3, Side4]
    #Each Side is of the form [e,[a0,a1,...,ak]]
    for i in range(4):
        SideA = Corona[i]
        SideB = Corona[(i+1)%4] #next side, wrapping around
        a_last = SideA[1][len(SideA[1])-1]
        a_first = SideB[1][0]
        exponentB = SideB[0]
        if a_last == a_first and exponentB == a_first:
            return(False) #fails unilaterality check
    return(True) #passes unilaterality check


#2. Vortex condition check: The vortex condition must be satisfied at each corner of the corona.

#First case: There is a 1 or 2 at the end of one side connecting to the beginning of the next side.

#This can be checked by looking at the last square of one side and the first square of the next side.
#If the last square of the first side is a size 1 or size 2 square and the exponent of the next side is more than 1 or 2 respectively, then this violates the vortex condition.
#Make sure that it also checks the last side connecting to the first side.

def Corona_Vortex_Condition_Check(Corona):
    #Corona is of the form [Side1, Side2, Side3, Side4]
    #Each Side is of the form [e,[a0,a1,...,ak]]
    for i in range(4):
        SideA = Corona[i]
        SideB = Corona[(i+1)%4] #next side, wrapping around
        a_last = SideA[1][len(SideA[1])-1]
        exponentB = SideB[0]
        if a_last == 1 and exponentB > 1:
            return(False) #fails vortex condition check
        if a_last == 2 and exponentB > 2:
            return(False) #fails vortex condition check
    return(True) #passes vortex condition check

#Second case:

# There is a 2 at the beginning of one side connecting to the end of the previous side.

#This can be checked by looking at the size of the last square of one side and looking at the first square of the next side.
#If the first square of the next side is a size 2 square and the last square of the previous side is a larger than a size 1 square, then this violates the vortex condition.
#Make sure that it also checks the last side connecting to the first side.
def Corona_Vortex_Condition_Check_Second_Case(Corona):
    #Corona is of the form [Side1, Side2, Side3, Side4]
    #Each Side is of the form [e,[a0,a1,...,ak]]
    for i in range(4):
        SideA = Corona[i]
        SideB = Corona[(i+1)%4] #next side, wrapping around
        a_first = SideB[1][0]
        a_last = SideA[1][len(SideA[1])-1]
        if a_first == 2 and a_last > 1:
            return(False) #fails vortex condition check
    return(True) #passes vortex condition check





In [44]:
#Testing the corona filters
TestCorona1 = [[0,[3,4]],[4,[4,3]],[0,[3,4]],[4,[4,3]]]
TestCorona2 = [[0,[3,2]],[2,[4,3]],[0,[3,4]],[4,[4,3]]]
TestCorona3 = [[0,[3,1]],[2,[4,3]],[0,[3,2]],[4,[4,3]]]
TestCorona4 = [[0,[3,2]],[3,[4,3]],[0,[3,4]],[4,[4,3]]]
TestCorona5 = [[0,[3,4]],[1,[2,3]],[0,[3,2]],[2,[4,3]]]
TestCorona6 = [[0,[3,4]],[2,[2,3]],[0,[3,1]],[2,[4,3]]]

#Check TestCorona1,2, and 3 with both filters
print("Testing Corona 1:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona1))
print("Vortex Condition Check: ", Corona_Vortex_Condition_Check(TestCorona1))
print("\nTesting Corona 2:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona2))
print("Vortex Condition Check: ", Corona_Vortex_Condition_Check(TestCorona2))
print("\nTesting Corona 3:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona3))
print("Vortex Condition Check: ", Corona_Vortex_Condition_Check(TestCorona3))   
print("\nTesting Corona 4:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona4))
print("Vortex Condition Check: ", Corona_Vortex_Condition_Check(TestCorona4))

#Check TestCorona5 and TestCorona6 with the first and second vortex condition filter
print("\nTesting Corona 5:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona5))
print("Vortex Condition Check (First Case): ", Corona_Vortex_Condition_Check(TestCorona5))
print("Vortex Condition Check (Second Case): ", Corona_Vortex_Condition_Check_Second_Case(TestCorona5))
print("\nTesting Corona 6:")
print("Unilaterality Check: ", Corona_Unilaterality_Check(TestCorona6))
print("Vortex Condition Check (First Case): ", Corona_Vortex_Condition_Check(TestCorona6))
print("Vortex Condition Check (Second Case): ", Corona_Vortex_Condition_Check_Second_Case(TestCorona6))

Testing Corona 1:
Unilaterality Check:  False
Vortex Condition Check:  True

Testing Corona 2:
Unilaterality Check:  False
Vortex Condition Check:  True

Testing Corona 3:
Unilaterality Check:  True
Vortex Condition Check:  False

Testing Corona 4:
Unilaterality Check:  False
Vortex Condition Check:  False

Testing Corona 5:
Unilaterality Check:  True
Vortex Condition Check (First Case):  True
Vortex Condition Check (Second Case):  False

Testing Corona 6:
Unilaterality Check:  True
Vortex Condition Check (First Case):  False
Vortex Condition Check (Second Case):  False


## Filters on the lists of coronas

In [ ]:
#Corona lists filters:
# These are filters that will be applied to the list of all possible coronas after stitching together four sides.

#1. Remove duplicate coronas that are rotations of each other
def Remove_Rotational_Duplicates(CoronaList):
    UniqueCoronaList = []
    for Corona in CoronaList:
        #Check if any rotation of Corona is already in UniqueCoronaList
        is_duplicate = False
        for i in range(4):
            RotatedCorona = [Corona[(j+i)%4] for j in range(4)]
            if RotatedCorona in UniqueCoronaList:
                is_duplicate = True
                break
        if is_duplicate == False:
            UniqueCoronaList.append(Corona)
    return(UniqueCoronaList)

#2. Remove duplicate coronas that are reflections of each other
def Remove_Reflectional_Duplicates(CoronaList):
    UniqueCoronaList = []
    for Corona in CoronaList:
        #Check if any reflection of Corona is already in UniqueCoronaList
        is_duplicate = False
        ReflectedCorona = [Corona[0], Corona[3], Corona[2], Corona[1]] #simple reflection across vertical axis
        for i in range(4):
            RotatedReflectedCorona = [ReflectedCorona[(j+i)%4] for j in range(4)]
            if RotatedReflectedCorona in UniqueCoronaList:
                is_duplicate = True
                break
        if is_duplicate == False:
            UniqueCoronaList.append(Corona)
    return(UniqueCoronaList)

In [19]:
#Testing the corona filters
CoronaList = [
    [[0,[3,4]],[4,[4,3]],[2,[3,2]],[0,[4,3]]],
    [[0,[3,4]],[1,[2,3]],[0,[4,3]],[2,[3,2]]],
    [[1,[2,3]],[0,[4,3]],[2,[3,2]],[0,[3,4]]],
    [[0,[4,3]],[2,[3,2]],[0,[3,4]],[1,[2,3]]],
    [[2,[3,2]],[0,[3,4]],[1,[2,3]],[0,[4,3]]],
    [[0,[2,3]],[1,[3,4]],[0,[3,2]],[2,[4,3]]],
    [[0,[4,3]], [1,[2,3]], [0,[3,4]], [2,[3,2]]]
]   
#Print original corona list
print("Original Corona List:")
for Corona in CoronaList:
    print(Corona)

#Test corona list filters
FilteredCoronaList = Corona_List_Filters(CoronaList)
print("\nFiltered Corona List (after unilaterality check):")
for Corona in FilteredCoronaList:
    print(Corona) 

#Test removing rotational duplicates
UniqueCoronaList_Rot = Remove_Rotational_Duplicates(FilteredCoronaList)
print("\nUnique Corona List (after removing rotational duplicates):")
for Corona in UniqueCoronaList_Rot:
    print(Corona)

#Test removing reflectional duplicates
UniqueCoronaList_Ref = Remove_Reflectional_Duplicates(UniqueCoronaList_Rot)
print("\nUnique Corona List (after removing reflectional duplicates):")
for Corona in UniqueCoronaList_Ref:
    print(Corona)


Original Corona List:
[[0, [3, 4]], [4, [4, 3]], [2, [3, 2]], [0, [4, 3]]]
[[0, [3, 4]], [1, [2, 3]], [0, [4, 3]], [2, [3, 2]]]
[[1, [2, 3]], [0, [4, 3]], [2, [3, 2]], [0, [3, 4]]]
[[0, [4, 3]], [2, [3, 2]], [0, [3, 4]], [1, [2, 3]]]
[[2, [3, 2]], [0, [3, 4]], [1, [2, 3]], [0, [4, 3]]]
[[0, [2, 3]], [1, [3, 4]], [0, [3, 2]], [2, [4, 3]]]
[[0, [4, 3]], [1, [2, 3]], [0, [3, 4]], [2, [3, 2]]]

Filtered Corona List (after unilaterality check):
[[0, [3, 4]], [1, [2, 3]], [0, [4, 3]], [2, [3, 2]]]
[[1, [2, 3]], [0, [4, 3]], [2, [3, 2]], [0, [3, 4]]]
[[0, [4, 3]], [2, [3, 2]], [0, [3, 4]], [1, [2, 3]]]
[[2, [3, 2]], [0, [3, 4]], [1, [2, 3]], [0, [4, 3]]]
[[0, [2, 3]], [1, [3, 4]], [0, [3, 2]], [2, [4, 3]]]
[[0, [4, 3]], [1, [2, 3]], [0, [3, 4]], [2, [3, 2]]]

Unique Corona List (after removing rotational duplicates):
[[0, [3, 4]], [1, [2, 3]], [0, [4, 3]], [2, [3, 2]]]
[[0, [2, 3]], [1, [3, 4]], [0, [3, 2]], [2, [4, 3]]]
[[0, [4, 3]], [1, [2, 3]], [0, [3, 4]], [2, [3, 2]]]

Unique Corona List

## Master Filtering Function

In [46]:
#Master Filter
#This function will take in a list of coronas and apply all the corona filters to it.

def Master_Corona_Filter(CoronaList):
    #First apply the unilaterality and vortex condition checks
    FilteredCoronaList = []
    for Corona in CoronaList:
        #Apply unilaterality check
        if Corona_Unilaterality_Check(Corona) == True:
            FilteredCoronaList.append(Corona)
        #Apply vortex condition check
        elif Corona_Vortex_Condition_Check(Corona) == True:
            FilteredCoronaList.append(Corona)
        #Apply second case of vortex condition check
        elif Corona_Vortex_Condition_Check_Second_Case(Corona) == True:
            FilteredCoronaList.append(Corona)
    #Next remove rotational and reflectional duplicates
    UniqueCoronaList_Rot = Remove_Rotational_Duplicates(FilteredCoronaList)
    UniqueCoronaList_Ref = Remove_Reflectional_Duplicates(UniqueCoronaList_Rot)
    return(UniqueCoronaList_Ref)



# Stitching Together the Coronas

In [20]:
#Stitching together sides to get full coronas:
#1. put all the possibilities generated by each of the four side type generators into four different lists
#This will be all the possible variations for each side. Make it a function where you input the center size and max square size.
def Generate_All_Sides(C,m):
    SideTypei = Side_Type_i_Generator(C,m)
    SideTypeii = Side_Type_ii_Generator(C,m)
    SideTypeiii = Side_Type_iii_Generator(C,m)
    SideTypeiv = Side_Type_iv_Generator(C,m)
    SideTypev = Side_Type_v_Generator(C,m)
    SideTypevi = Side_Type_vi_Generator(C,m)
    return(SideTypei, SideTypeii, SideTypeiii, SideTypeiv, SideTypev, SideTypevi)


In [21]:
Generate_All_Sides(4,4)

([[0, [1, 4]], [0, [3, 2]], [0, [2, 3]], [0, [2, 4]], [0, [3, 4]]],
 [[1, [4, 1]], [1, [3, 2]], [1, [2, 3]], [2, [4, 2]], [3, [4, 3]]],
 [[0, [3, 1]], [0, [1, 3]]],
 [[1, [4, 2]],
  [1, [2, 4]],
  [1, [4, 3]],
  [2, [4, 3]],
  [1, [3, 4]],
  [2, [3, 4]]],
 [[4, [4, 3, 1]]],
 [[4, [4, 3, 2]], [4, [4, 3, 4]]])

In [22]:
#2.Now we want to stitch together sides from these lists to get full coronas:
#But we want to categorize the full coronas based on the types of sides used, we will have them archetypes
#Use this key to categorize:

#archetype 6: (vi, iii, vi, iii)
#archetype 7: (v, ii, vi, iii)
#archetype 8: (v, ii, ii, ii)
#archetype 9: (vi, iii, ii, ii)
#archetype 10: (v, iv, iii, ii)
#archetype 11: (v, ii, iv, iii)
#archetype 12: (i, i, i, i)
#archetype 13: (i, iii, iv, i)
#archetype 14: (i, iii, ii, iv)
#archetype 15: (iv, iii, iv, iii)

## Archetype VI

In [48]:
#Archetype 6 generator:
#Generate all coronas of the form (vi, iii, vi, iii) and puts them in a list 

def Archetype_6_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypevi = Sides[5]
    SideTypeiii = Sides[2]
    FullCoronas = []
    X = it.product(SideTypevi, SideTypeiii, SideTypevi, SideTypeiii)
    for i in X:
        FullCoronas.append(i)
    #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)


In [51]:
Archetype_6_Generator(4,4)

[([4, [4, 3, 2]], [0, [3, 1]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 2]], [0, [1, 3]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 2]], [0, [1, 3]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 2]], [0, [1, 3]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 2]], [0, [1, 3]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 4]], [0, [3, 1]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 4]], [0, [3, 1]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 4]], [0, [3, 1]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 4]], [0, [3, 1]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 4]], [0, [1, 3]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 4]], [0, [1, 3]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 4]], [0, [1, 3]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 4]], [0, [1, 3]], [4, [4, 3, 4]], [0, [1, 3]])]

## Archetype VII

In [49]:
#Archetype 7 generator:
#Generates all coronas of the form (v, ii, vi, iii) and puts them in a list
def Archetype_7_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypev = Sides[4]
    SideTypeii = Sides[1]
    SideTypevi = Sides[5]
    SideTypeiii = Sides[2]
    FullCoronas = []
    X = it.product(SideTypev, SideTypeii, SideTypevi, SideTypeiii)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)


In [50]:
Archetype_7_Generator(4,4)

[([4, [4, 3, 1]], [1, [4, 1]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [2, 3]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [2, 3]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [2, 3]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [2, 3]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [2, [4, 2]], [4, [4, 3, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [2, [4, 2]], [4, [4, 3, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [2, [4, 2]], [4, [4, 3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [2, [4, 2]], [4, [4, 3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [3, [4

## Archetype VIII

In [52]:
#Archetype 8 generator:
#Generates all coronas of the form (v, ii, ii, ii) and puts them in a list
def Archetype_8_Generator(C,m): 
    Sides = Generate_All_Sides(C,m)
    SideTypev = Sides[4]
    SideTypeii = Sides[1]
    FullCoronas = []
    X = it.product(SideTypev, SideTypeii, SideTypeii, SideTypeii)
    for i in X:
        FullCoronas.append(i)#Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)
    

In [53]:
Archetype_8_Generator(4,4)


[([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 1]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 1]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 1]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 1]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 1]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 2]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 2]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 2]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 2]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 2]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 3]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 3]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 3]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 3]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 3]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [4, 2]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [4, 2]], [1, [3, 2]]),
 ([4, [4, 3, 1

## Archetype IX

In [55]:
#Archetype 9 generator:
#Generates all coronas of the form (vi, iii, ii, ii) and puts them in a list
def Archetype_9_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypevi = Sides[5]
    SideTypeiii = Sides[2]
    SideTypeii = Sides[1]
    FullCoronas = []
    X = it.product(SideTypevi, SideTypeiii, SideTypeii, SideTypeii)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [56]:
Archetype_9_Generator(4,4)

[([4, [4, 3, 2]], [0, [3, 1]], [1, [4, 1]], [1, [4, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [4, 1]], [1, [3, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [4, 1]], [1, [2, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [4, 1]], [2, [4, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [4, 1]], [3, [4, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [3, 2]], [1, [4, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [3, 2]], [1, [3, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [3, 2]], [1, [2, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [3, 2]], [2, [4, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [3, 2]], [3, [4, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [2, 3]], [1, [4, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [2, 3]], [1, [3, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [2, 3]], [1, [2, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [2, 3]], [2, [4, 2]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [1, [2, 3]], [3, [4, 3]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [2, [4, 2]], [1, [4, 1]]),
 ([4, [4, 3, 2]], [0, [3, 1]], [2, [4, 2]], [1, [3, 2]]),
 ([4, [4, 3, 2

## Archetype X

In [57]:
#Archetype 10 generator:
#Generates all coronas of the form (v, iv, iii, ii) and puts them in a list
def Archetype_10_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypev = Sides[4]
    SideTypeiv = Sides[3]
    SideTypeiii = Sides[2]
    SideTypeii = Sides[1]
    FullCoronas = []
    X = it.product(SideTypev, SideTypeiv, SideTypeiii, SideTypeii)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [58]:
Archetype_10_Generator(4,4)

[([4, [4, 3, 1]], [1, [4, 2]], [0, [3, 1]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [3, 1]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [3, 1]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [3, 1]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [3, 1]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [1, 3]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [1, 3]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [1, 3]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [1, 3]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [4, 2]], [0, [1, 3]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [3, 1]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [3, 1]], [1, [3, 2]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [3, 1]], [1, [2, 3]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [3, 1]], [2, [4, 2]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [3, 1]], [3, [4, 3]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [1, 3]], [1, [4, 1]]),
 ([4, [4, 3, 1]], [1, [2, 4]], [0, [1, 3]], [1, [3, 2]]),
 ([4, [4, 3, 1

## Archetype XI

In [59]:
#Archetype 11 generator:
#Generates all coronas of the form (v, ii, iv, iii) and puts them in a list
def Archetype_11_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypev = Sides[4]
    SideTypeii = Sides[1]
    SideTypeiv = Sides[3]
    SideTypeiii = Sides[2]
    FullCoronas = []
    X = it.product(SideTypev, SideTypeii, SideTypeiv, SideTypeiii)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [60]:
Archetype_11_Generator(4,4)

[([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [2, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 3]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [4, 3]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [4, 3]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [4, 3]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [1, [3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [3, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [4, 1]], [2, [3, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [1, [4, 2]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [1, [4, 2]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [1, [2, 4]], [0, [3, 1]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [1, [2, 4]], [0, [1, 3]]),
 ([4, [4, 3, 1]], [1, [3, 2]], [1, [4, 3]], [0, [3, 1]]),
 ([4, [4, 3, 1

## Archetype XII

In [61]:
#Archetype 12 generator:
#Generates all coronas of the form (i, i, i, i) and puts them in a list
def Archetype_12_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypei = Sides[0]
    FullCoronas = []
    X = it.product(SideTypei, SideTypei, SideTypei, SideTypei)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [62]:
Archetype_12_Generator(1,4)

[([0, [2]], [0, [2]], [0, [2]], [0, [2]]),
 ([0, [2]], [0, [2]], [0, [2]], [0, [3]]),
 ([0, [2]], [0, [2]], [0, [2]], [0, [4]]),
 ([0, [2]], [0, [2]], [0, [3]], [0, [2]]),
 ([0, [2]], [0, [2]], [0, [3]], [0, [3]]),
 ([0, [2]], [0, [2]], [0, [3]], [0, [4]]),
 ([0, [2]], [0, [2]], [0, [4]], [0, [2]]),
 ([0, [2]], [0, [2]], [0, [4]], [0, [3]]),
 ([0, [2]], [0, [2]], [0, [4]], [0, [4]]),
 ([0, [2]], [0, [3]], [0, [2]], [0, [2]]),
 ([0, [2]], [0, [3]], [0, [2]], [0, [3]]),
 ([0, [2]], [0, [3]], [0, [2]], [0, [4]]),
 ([0, [2]], [0, [3]], [0, [3]], [0, [2]]),
 ([0, [2]], [0, [3]], [0, [3]], [0, [3]]),
 ([0, [2]], [0, [3]], [0, [3]], [0, [4]]),
 ([0, [2]], [0, [3]], [0, [4]], [0, [2]]),
 ([0, [2]], [0, [3]], [0, [4]], [0, [3]]),
 ([0, [2]], [0, [3]], [0, [4]], [0, [4]]),
 ([0, [2]], [0, [4]], [0, [2]], [0, [2]]),
 ([0, [2]], [0, [4]], [0, [2]], [0, [3]]),
 ([0, [2]], [0, [4]], [0, [2]], [0, [4]]),
 ([0, [2]], [0, [4]], [0, [3]], [0, [2]]),
 ([0, [2]], [0, [4]], [0, [3]], [0, [3]]),
 ([0, [2]],

## Archetype XIII

In [63]:
#Archetype 13 generator:
#Generates all coronas of the form (i, iii, iv, i) and puts them in a list
def Archetype_13_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypei = Sides[0]
    SideTypeiii = Sides[2]
    SideTypeiv = Sides[3]
    FullCoronas = []
    X = it.product(SideTypei, SideTypeiii, SideTypeiv, SideTypei)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [64]:
Archetype_13_Generator(4,4)

[([0, [1, 4]], [0, [3, 1]], [1, [4, 2]], [0, [1, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 2]], [0, [3, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 2]], [0, [2, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 2]], [0, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 2]], [0, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 4]], [0, [1, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 4]], [0, [3, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 4]], [0, [2, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 4]], [0, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 4]], [0, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 3]], [0, [1, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 3]], [0, [3, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 3]], [0, [2, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 3]], [0, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 3]], [0, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [2, [4, 3]], [0, [1, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [2, [4, 3]], [0, [3, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [2, [4, 3]], [0, [2, 3]]),
 ([0, [1, 

## Archetype XIV

In [65]:
#Archetype 14 generator:
#Generates all coronas of the form (i, iii, ii, iv) and puts them in a list
def Archetype_14_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypei = Sides[0]
    SideTypeiii = Sides[2]
    SideTypeii = Sides[1]
    SideTypeiv = Sides[3]
    FullCoronas = []
    X = it.product(SideTypei, SideTypeiii, SideTypeii, SideTypeiv)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [66]:
Archetype_14_Generator(4,4)

[([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [1, [4, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [1, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [1, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [2, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [1, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [4, 1]], [2, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [1, [4, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [1, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [1, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [2, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [1, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [3, 2]], [2, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [1, [4, 2]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [1, [2, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [1, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [2, [4, 3]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [1, [3, 4]]),
 ([0, [1, 4]], [0, [3, 1]], [1, [2, 3]], [2, [3, 4]]),
 ([0, [1, 

## Archetype XV

In [67]:
#Archetype 15 generator:
#Generates all coronas of the form (iv, iii, iv, iii) and puts them in a list
def Archetype_15_Generator(C,m):
    Sides = Generate_All_Sides(C,m)
    SideTypeiv = Sides[3]
    SideTypeiii = Sides[2]
    FullCoronas = []
    X = it.product(SideTypeiv, SideTypeiii, SideTypeiv, SideTypeiii)
    for i in X:
        FullCoronas.append(i)
        #Apply the master corona filter to FullCoronas
    FinalList = Master_Corona_Filter(FullCoronas)
    return(FinalList)

In [68]:
Archetype_15_Generator(4,4)

[([1, [4, 2]], [0, [3, 1]], [1, [4, 2]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [4, 2]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [2, 4]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [2, 4]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [4, 3]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [4, 3]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [3, 1]], [2, [4, 3]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [2, [4, 3]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [3, 4]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [1, [3, 4]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [3, 1]], [2, [3, 4]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [3, 1]], [2, [3, 4]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [4, 2]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [4, 2]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [2, 4]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [2, 4]], [0, [1, 3]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [4, 3]], [0, [3, 1]]),
 ([1, [4, 2]], [0, [1, 3]], [1, [4, 3]], [0, [1, 3]]),
 ([1, [4, 

# Drawing Coronas

In [ ]:
#This would ideally draw the each corona and put them in a file for later viewing.
#Ideally in that doc, you would have the following info:
#1. The drawing of each corona categorized by Archetype
#2. A clear labeling of each corona with its side sequences and exponents
#3. A count of how many unique coronas there are per archetype and in total
#4. A title that follows the format "Coronas with Center Size C and Max Square Size m"
#5. Each size square would be drawn to scale based on its size and have a small label indicating its size
    #5.5 Each size square would have a different color and black borders to clearly see the boundaries
    #5.75 The center would have a distinct color than the other squares. 
#6. The coronas would be arranged neatly on each page with enough spacing to clearly see each one without overlap

# List of Coronas for each size

In [ ]:
#1. Combines all the archetype generators to produce a master list of coronas for each size from 1 to a given C value with a given m value.
#2. For each size from 1 to C, it generates all coronas for that size and stores them in a file using the drawing function 
    #with the size as the key.
#3. The final output is a comprehensive file containing all coronas for sizes 1 to C with max square size m, categorized by archetype and drawn for visualization.
